In [ ]:
def simulate_strategies_basic(agents, n_mc, n_steps, x0_sampler, **kwargs):

  x0 = x0_sampler(n_mc)

  s = x0[:,0]   
  results = list()
  for name, agent in agents.items():
    results.append(BasicResult(x0, agent))

  for i in range(1, n_steps):
    with torch.no_grad():
      # Here we update the state - update s as a geometric brownian motion
      s_step = partial(GBM_step, **kwargs)
      s_next = s_step(s=s)

      for result in results: 
        __, x, __, __ = result.agent.step(result.path[-1], n_mc=1, s_next = s_next, **kwargs)
            
        result.update_result(x)
      s = s_next
      
  return results

In [ ]:
def simulate_strategies(agents, n_mc, n_steps, x0_sampler, f_L=None, **kwargs):
  """
  Assumes that all agents had the same variable_fees bool

  Parameters
  ----------
    agents : dict
      a dictionary of agent names and agents
    n_mc : int
      the number of simulations to run
    n_steps : int
      number of steps to run the simulation for
    x0_sampler : partial
      the state space sampler to use (should be the same as used for agents)
    f_L : float, optional
      the liquidity fee to set in the case of a variable fee market. If None
      use fees from sampler
  """
    
  x0 = x0_sampler(n_mc)

  '''
  # Start with initial price equal to CPFM price
  x0[:,0] = x0[:,1] 
  s = x0[:,0]
  # Leave the pool price, x[:,1], and pool balance, x[:,2], as sampled
  # Start with 0 position
  x0[:,3] = torch.zeros_like(x0[:,3])
  # Leave the general account balance as samples 
  # TODO: sometimes general account is too low to trade anything
  # Start with 0 margin account
  x0[:,5] = torch.zeros_like(x0[:,5])

  # Manually set fee for market
  if list(agents.values())[0].variable_fees and f_L is not None:
    x0[:,6] = torch.ones_like(x0[:,6]) * f_L
  '''

  s = x0[:,0]   
  results = list()
  for name, agent in agents.items():
    if isinstance(agent, ActorCritic):
      agent.v.eval()
      agent.C.eval()
    results.append(AgentResult(x0, agent, name = name))
      
  for i in range(1, n_steps):
    kwargs['t'] = i * kwargs['tau']
    with torch.no_grad():
        
      if kwargs.get('increment_type') == 'Brownian Bridge':
        b_next = Brownian_bridge_step(**kwargs)
        kwargs['dW'] = b_next.clone() - kwargs['b'].clone()
        kwargs['b'] = b_next.clone()
      # Here we update the state - update s as a geometric brownian motion
      s_step = partial(GBM_step, **kwargs)
      s_next = s_step(s=s)

      for result in results: 
        if isinstance(result.agent, ActorCritic):
          a, x, u, profit_alpha = result.agent.step(result.path[-1], n_mc=1,
                                                    train=False, s_next = s_next, **kwargs)
        else:
          a, x, u, profit_alpha = result.agent.step(result.path[-1], n_mc=1, s_next = s_next, **kwargs)
            
        result.update_result(a, x, u, profit_alpha)
      
      s = s_next
    
  for result in results:
      result.finalize_result()
      
  return results

In [ ]:
def simulate_strategies_var_fees(agents, n_mc, n_steps, x0_sampler, fees, **kwargs):
  """
  Assumes that all agents had the same variable_fees bool

  Parameters
  ----------
    agents : dict
      a dictionary of agent names and agents
    n_mc : int
      the number of simulations to run
    n_steps : int
      number of steps to run the simulation for
    x0_sampler : partial
      the state space sampler to use (should be the same as used for agents)
    f_L : float, optional
      the liquidity fee to set in the case of a variable fee market. If None
      use fees from sampler
  """
    
  x0 = x0_sampler(n_mc)

  '''
  # Start with initial price equal to CPFM price
  x0[:,0] = x0[:,1] 
  s = x0[:,0]
  # Leave the pool price, x[:,1], and pool balance, x[:,2], as sampled
  # Start with 0 position
  x0[:,3] = torch.zeros_like(x0[:,3])
  # Leave the general account balance as samples 
  # TODO: sometimes general account is too low to trade anything
  # Start with 0 margin account
  x0[:,5] = torch.zeros_like(x0[:,5])

  # Manually set fee for market
  if list(agents.values())[0].variable_fees and f_L is not None:
    x0[:,6] = torch.ones_like(x0[:,6]) * f_L
  '''

  s = x0[:,0]   
  results = list()

  for fee in fees:
    x0[:,6] = fee*torch.ones_like(x0[:,6])   
    for name, agent in agents.items():
      if isinstance(agent, ActorCritic):
        agent.v.eval()
        agent.C.eval()
        print(x0[0,6])
      results.append(AgentResult(x0.clone(), agent, name = name + str(fee)))
      
  for i in range(1, n_steps):
    kwargs['t'] = i * kwargs['tau']
    with torch.no_grad():
        
      if kwargs.get('increment_type') == 'Brownian Bridge':
        b_next = Brownian_bridge_step(**kwargs)
        kwargs['dW'] = b_next.clone() - kwargs['b'].clone()
        kwargs['b'] = b_next.clone()
      # Here we update the state - update s as a geometric brownian motion
      s_step = partial(GBM_step, **kwargs)
      s_next = s_step(s=s)

      for result in results: 
        if isinstance(result.agent, ActorCritic):
          a, x, u, profit_alpha = result.agent.step(result.path[-1], n_mc=1,
                                                    train=False, s_next = s_next, **kwargs)
        else:
          a, x, u, profit_alpha = result.agent.step(result.path[-1], n_mc=1, s_next = s_next, **kwargs)
            
        result.update_result(a, x, u, profit_alpha)
      
      s = s_next
    
  for result in results:
      result.finalize_result()
      
  return results

In [ ]:
class BasicResult():

  def __init__(self, x0, agent, name=''):
    self.path = [x0]
    self.name = name
    self.agent = agent

  def update_result(self, x):
    self.path.append(x)

In [ ]:
class AgentResult():
  
  def __init__(self, x0, agent, name = ''):
      
    self.path = [x0]
    self.actions = []
    self.PnL = []
    self.utility = []
    self.name = name
    self.agent = agent
      
  def update_result(self, a, x, u, profit_alpha):
    self.PnL.append(profit_alpha)
    self.path.append(x)
    self.actions.append(a)
    self.utility.append(u.reshape(-1))
      
  def finalize_result(self):
    path = torch.stack(self.path, 1)
    self.s, self.price, self.sum, self.n, self.g, self.m=\
        path[...,0].cpu(), path[...,1].cpu(), path[...,2].cpu(),\
        path[...,3].cpu(), path[...,4].cpu(), path[...,5].cpu()
    if self.agent.variable_fees:
      self.f_L = path[...,6].cpu()
    self.A_S = self.sum / (1 + self.price)
    self.A_B = self.sum * self.price / (1 + self.price)
    self.V = self.A_B + self.s*self.A_S # (n_mc, L)
    self.actions = torch.stack(self.actions, 1) # (n_mc, L, 3)
    self.PnL = torch.stack(self.PnL,1) # (n_mc, L)
    self.utility = torch.stack(self.utility, 1) # (n_mc, L)
    self.c = self.actions[...,-3:]
    self.PnL_cumsum = torch.cumsum(self.PnL, 1)
  
  def final_profits(self):
    return torch.sum(self.PnL,1)

  def get_mean_pnl(self):
    mean_pnl = torch.mean(self.PnL_cumsum, 0)
    std_pnl = torch.std(self.PnL_cumsum, 0)

    return mean_pnl, std_pnl

  def get_pnl_quantile(self, percent):
    low_percent = (1. - percent) / 2.
    high_percent = percent + (1. - percent) / 2.
    low = torch.quantile(self.PnL_cumsum, low_percent, 0)
    high = torch.quantile(self.PnL_cumsum, high_percent, 0)
    return low, high

  def get_start_of_step_price_differential(self):
    return self.s - self.price
  
  def get_end_of_step_price_differential(self):
    return self.s[:,0:-2] - self.price[:,1:-1]

In [ ]:
LZ_COLOURWHEEL = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
                  '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']


def plot_inventory(ax, agent, A_B, A_S, n, g, m):
    ax.plot(g, label=r'General Account')
    ax.plot(m, label=r'Margin Account')
    ax.plot(torch.where(n > 0, n * agent.ts * agent.M_long(A_B, A_S) * agent.l_s,
                        -n * agent.ts * agent.M_short(A_B, A_S) * agent.l_s),
            '--', label=r'Search Margin')
    ax.plot(torch.where(n > 0,  n * agent.ts * agent.M_long(A_B, A_S) * agent.l_r,
                        -n * agent.ts * agent.M_short(A_B, A_S) * agent.l_r),
            '--', label=r'Release Margin')
    ax.set_title(r'Inventory')
    ax.set_xlabel('Step')
    ax.set_ylabel('Value')
    ax.legend()
    ax.grid()

def plot_inventory_multi(ax, i, results, multi=False):
  j = 0
  
  colours = plt.rcParams['axes.prop_cycle'].by_key()['color']
  j = 0
  for result in results:
    if multi:
      ax.plot(result.g[i], '-', label=result.name , # + ' General Account'
              color = colours[j])
      ax.plot(result.m[i], '--', #label=result.name ,# + ' Margin Account',
              color = colours[j])
      j += 1

    else:
      ax.plot(result.g[i], '-', label=' General Account')
      ax.plot(result.m[i], '--', label=' Margin Account')
      ax.plot(torch.where(result.n[i] > 0,
                          result.n[i] * result.agent.ts * result.agent.M_long(result.A_B[i], result.A_S[i]) * result.agent.l_s,
                          -result.n[i] * result.agent.ts * result.agent.M_short(result.A_B[i], result.A_S[i]) * result.agent.l_s),
              '--', label='Search Margin')
      ax.plot(torch.where(result.n[i] > 0,
                          result.n[i] * result.agent.ts * result.agent.M_long(result.A_B[i], result.A_S[i]) * result.agent.l_r,
                          -result.n[i] * result.agent.ts * result.agent.M_short(result.A_B[i], result.A_S[i]) * result.agent.l_r),
              '--', label='Release Margin')

  ax.set_title(r'Inventory')
  ax.set_xlabel('Step')
  ax.set_ylabel('Account Balance')
  ax.legend()
  ax.grid()


def plot_pnl_multi(ax, i, results, multi=False):

    for result in results:
      PnL = result.PnL.clone()[i].cumsum(0).cpu() 
      ax.plot(PnL, label = result.name)

    ax.set_title('Cumulative PNL')
    ax.set_xlabel('Step')
    ax.set_ylabel('Cumulative PNL')
    if multi:
      ax.legend()
    ax.grid()

def plot_action_multi(ax, i, results, multi=False):
  labels = ['Long', 'Short', 'Do Nothing']
  for result in results:
    for j in range(3):
      ax.plot(result.actions[i,:,j].numpy(), '--', label=labels[j])
  
  ax.set_title('Actions')
  ax.set_xlabel('Step')
  ax.set_ylabel(r'Action')
  ax.legend()
  ax.grid()    

def plot_action_multi(ax, i, results, multi=False):
  labels = ['Long', 'Short', 'Do Nothing']
  for result in results:
    for j in range(3):
      ax.plot(result.actions[i,:,j].numpy(), '--', label=labels[j])
  
  ax.set_title('Actions')
  ax.set_xlabel('Step')
  ax.set_ylabel(r'Action')
  ax.legend()
  ax.grid() 

def plot_price_multi(ax, i, results, multi=False, colours=LZ_COLOURWHEEL):

  j = 0


  ax.plot(results[0].s[i], label = 'Reference Price', color = 'black')

  for result in results:
    A_B = result.A_B[i]
    A_S = result.A_S[i]
    ax.plot(A_B/ A_S, '--', color = colours[j], label = result.name + ' Price')
    j += 1

  ax.set_title('Future Price')
  ax.set_xlabel('Step')
  ax.set_ylabel(r'Price')
  ax.legend()
  ax.grid() 

def plot_units_held_multi(ax, i, results, multi=False):
  for result in results:
    ax.plot(result.n[i].numpy(), '.--', label=result.name)

  ax.set_title('Agent Position')
  ax.set_xlabel('Step')
  ax.set_ylabel('Position')
  if multi:
    ax.legend()
  ax.grid()

def make_comparison_plots(agent_results, n_mc, indicies=[], save_fig=False):

  for j in range(n_mc):
    if len(indicies) <= j:
      i = j
    else:
      i = indicies[j]

    print(i)
            
    fig, ax = plt.subplots(1, 4, figsize=(20, 4))
    
    plot_units_held_multi(ax[0], i, agent_results, multi=True)
    
    #plot_action(ax[0], actions[i])
    
    #plot_pnl(ax[1], PnL[i])

    '''
    for key, value in agent_results.items():
        ax[1].plot(value.value()[i], label=key)
        ax[1].legend()
    '''

    plot_pnl_multi(ax[1], i, agent_results, multi=True)
        
    plot_price_multi(ax[2], i, agent_results, multi=True)
        
    plot_inventory_multi(ax[3], i, agent_results, multi=True)
    
    fig.tight_layout()
    if save_fig:
      fig.savefig(os.path.join(results_path,str(i)+fig_name))
    plt.show()

def make_comparison_plots_square(agent_results, n_mc, indicies=[], save_fig=False):

  for j in range(n_mc):
    if len(indicies) <= j:
      i = j
    else:
      i = indicies[j]

    print(i)
            
    fig, ax = plt.subplots(2, 2, figsize=(15, 10))
    
    plot_units_held_multi(ax[0,0], i, agent_results, multi=True)
    ax[0,0].set_title('(a)')

    plot_pnl_multi(ax[0,1], i, agent_results, multi=True)
    ax[0,1].set_title('(b)')
        
    plot_price_multi(ax[1,0], i, agent_results, multi=True)
    ax[1,0].set_title('(c)')
        
    plot_inventory_multi(ax[1,1], i, agent_results, multi=True)
    ax[1,1].set_title('(d)')
    
    fig.tight_layout()
    if save_fig:
      fig.savefig(os.path.join(results_path,str(i)+fig_name))
    plt.show()

In [ ]:
def add_ideal_agents(agents_dict, parameters_dict, same_conditions=False):

  if same_conditions:
    parameters = list(parameters_dict.values())[0]
    agent = list(agents_dict.values())[0]

    if parameters['market_args']['random_agent_update']:
      ideal_agent = IdealOneStepAgentTraders(utility = agent.f, device = device, **parameters['market_args'])
    else:
      ideal_agent = IdealOneStepAgent(utility =agent.f, device=device, **parameters['market_args'])

    agents_dict['Ideal Agent'] = ideal_agent
  else:
    for name, parameters in parameters_dict.items():
      agent = agents_dict[name]
      if parameters['market_args']['random_agent_update']:
        ideal_agent = IdealOneStepAgentTraders(utility = agent.f, device = device, **parameters['market_args'])
      else:
        ideal_agent = IdealOneStepAgent(utility =agent.f, device=device, **parameters['market_args'])

      agents_dict[name +' Ideal Agent'] = ideal_agent

  return agents_dict